In [47]:
import os 
import pandas as pd
import numpy as np
from pathlib import Path

data_path = Path('../../../data/annotations/group_mention_categorization')

### Load previous annotations

In [48]:
fp = data_path / 'final_annotations.tsv'
annotations = pd.read_csv(fp, sep='\t')

In [49]:
annotations.q_id.value_counts()

q_id
non-economic_attributes    6600
economic_attributes        4200
Name: count, dtype: int64

In [50]:
annotations.groupby(['mention_id', 'attribute_combination']).filter(lambda x: x.label.nunique() > 1)
# NOTE: should be none !!!

,mention_id,text,mention,prev_texts,next_texts,q_id,q_category,category,round,attribute,attribute_combination,label


In [51]:
annotations['attribute_combination_val'] = annotations['attribute_combination'].str.replace(': ', '__').str.replace('non-', 'non').str.replace(r'[^a-z_]+', '_', regex=True)

### Load susbet of reviewed and corrected annotations

In [52]:
fp = data_path / 'attribute-misclassification-review' / 'reviewed.csv'
reviewed = pd.read_csv(fp)
# NOTE: only values for seleceted attribute combinations have been reviewed by expert - but the data has all binary indicators in long format

In [53]:
# pivot data in columns starting with "economic__" or "noneconomic__" to long format using mentio_id as index
reviewed_long = reviewed.melt(
    id_vars=['mention_id'], 
    value_vars=[col for col in reviewed.columns if col.startswith('economic__') or col.startswith('noneconomic__')],
    var_name='attribute_combination_val',
    value_name='corrected_label'
)

In [54]:
reviewed_long['corrected_label'] = reviewed_long.corrected_label.map({0: 'No', 1: 'Yes', np.nan: np.nan})

In [55]:
reviewed_long.value_counts('corrected_label')

corrected_label
No     2712
Yes     264
Name: count, dtype: int64

## add manual recodings of econ class/ecology of group revisions

::: {.callout-note}

We decided to discard the economic attribute categories _class membership_ and _ecology of group_ because they are high-level sociological constructs that are on a different level of abstraction, latent, and thus difficult to reliably identify based on textual features alone.

:::

In [56]:
corrected_labels = """\
13229_199803-187861-3	ordinary people	economic__income_wealth_economic_status
13951_197702-181934-1	ordinary people	economic__income_wealth_economic_status
14820_197509-197462-1	ordinary people	economic__income_wealth_economic_status
51320_201706-244572-1	ordinary people	economic__income_wealth_economic_status
13230_198401-183437-1	bourgeois prophets	economic__income_wealth_economic_status
171101_201207-307846-2	social classes	economic__income_wealth_economic_status
51320_199204-226480-1	social classes	economic__income_wealth_economic_status
21111_199111-31476-4	social class	economic__income_wealth_economic_status
23112_198406-290795-4	classes of society	economic__income_wealth_economic_status
86110_201404-269177-1	special members of the (upper) middle classes	economic__income_wealth_economic_status
13951_197501-181621-1	workers	economic__occupation_profession; economic__employment_status
51320_201912-249608-1	working-class people	economic__income_wealth_economic_status; economic__occupation_profession; economic__employment_status
42110_198611-03946-1	Powerful cliques behind and within the old parties	economic__occupation_profession
63110_200410-00030-2	the world's highest per capita users of water	economic__other\
"""
corrected_labels_df = pd.DataFrame([row.split("\t") for row in corrected_labels.split("\n")], columns=['mention_id', 'mention', 'attribute_combination_val'])
del corrected_labels_df['mention']
corrected_labels_df['corrected_label'] = "Yes"
corrected_labels_df['attribute_combination_val'] = corrected_labels_df['attribute_combination_val'].str.split('; ')
corrected_labels_df = corrected_labels_df.explode('attribute_combination_val')

In [57]:
corrected_labels_df

,mention_id,attribute_combination_val,corrected_label
0,13229_199803-187861-3,economic__income_wealth_economic_status,Yes
1,13951_197702-181934-1,economic__income_wealth_economic_status,Yes
2,14820_197509-197462-1,economic__income_wealth_economic_status,Yes
3,51320_201706-244572-1,economic__income_wealth_economic_status,Yes
4,13230_198401-183437-1,economic__income_wealth_economic_status,Yes
5,171101_201207-307846-2,economic__income_wealth_economic_status,Yes
6,51320_199204-226480-1,economic__income_wealth_economic_status,Yes
7,21111_199111-31476-4,economic__income_wealth_economic_status,Yes
8,23112_198406-290795-4,economic__income_wealth_economic_status,Yes
9,86110_201404-269177-1,economic__income_wealth_economic_status,Yes


In [58]:
# NOTE: some corrected manually not in set of reviewed mentions
set(corrected_labels_df.mention_id.unique().tolist()).difference(set(reviewed_long.mention_id.unique().tolist()))

{'13230_198401-183437-1',
 '13951_197501-181621-1',
 '171101_201207-307846-2',
 '21111_199111-31476-4',
 '23112_198406-290795-4',
 '51320_199204-226480-1',
 '51320_201912-249608-1',
 '86110_201404-269177-1'}

In [60]:
# anti join
tmp = reviewed_long.merge(corrected_labels_df[['mention_id', 'attribute_combination_val']], how = 'outer', indicator = True)
tmp = tmp[tmp._merge == 'left_only'].drop('_merge', axis = 1)
tmp.query("mention_id=='13230_198401-183437-1'")

,mention_id,attribute_combination_val,corrected_label


In [61]:
reviewed_long = pd.concat([tmp, corrected_labels_df], ignore_index=True)

In [62]:
# NOTE: now they were added ✅
set(corrected_labels_df.mention_id.unique().tolist()).difference(set(reviewed_long.mention_id.unique().tolist()))

set()

In [63]:
reviewed_long.query("mention_id=='13230_198401-183437-1'")

,mention_id,attribute_combination_val,corrected_label
2975,13230_198401-183437-1,economic__income_wealth_economic_status,Yes


## merge

In [64]:
out = annotations.merge(reviewed_long, on=['mention_id', 'attribute_combination_val'], how='left', indicator=True)

### checl

In [ ]:
# NOTE: These should be none 🍏
tmp = out.groupby(['mention_id', 'attribute_combination_val']).size()
tmp[tmp > 1].reset_index(name='count')[['mention_id', 'attribute_combination_val']].merge(annotations, on=['mention_id', 'attribute_combination_val']).sort_values(['mention_id', 'attribute_combination'])[['mention_id', 'mention', 'attribute_combination_val', 'label']]

,mention_id,mention,attribute_combination_val,label


### Inspect impact of corrections

In [67]:
out.groupby("mention_id").agg({'_merge': 'first'}).value_counts('_merge')
# NOTE: 186 mentions have been reviewed in error analysis

_merge
left_only     414
both          186
right_only      0
Name: count, dtype: int64

In [68]:
out.query('_merge=="both"').groupby("mention_id")[['label', 'corrected_label']].apply(lambda x: (x.label != x.corrected_label).sum()).value_counts().sort_index()
# NOTE: this is the distribution of number of label changes per reviewed mention-in-context instance

0    140
1     47
2      5
3      2
Name: count, dtype: int64

In [69]:
out['label_change'] = out.apply(lambda x: np.nan if pd.isna(x.corrected_label) else f"{x.label} -> {x.corrected_label}" if x.label != x.corrected_label else np.nan, axis=1)
out.loc[~out.label_change.isna(), ["mention_id", "text", "mention",  "attribute_combination", "label_change"]].sort_values("mention_id")

,mention_id,text,mention,attribute_combination,label_change
80,11110_200609-393907-1,Everyone who is exposed to violence or threats...,Everyone who is exposed to violence or threats...,non-economic: crime,No -> Yes
656,11620_201009-396018-1,"In many situations, it is evident from the cir...",the woman who sells her body,non-economic: crime,Yes -> No
1039,12951_199709-332992-1,The Progress Party takes a strong distance fro...,"people based on race, religion and ethnic origin",non-economic: nationality,No -> Yes
1282,13229_199803-187861-3,"Far from the public light, few people in the b...",ordinary people,economic: income/wealth/economic status,No -> Yes
1408,13230_198401-183437-1,This is the truth behind the words of the many...,bourgeois prophets,economic: income/wealth/economic status,No -> Yes
...,...,...,...,...,...
9832,86710_201404-270610-2,"Together, these companies have deceived the ma...",those involved in the currency lending trap,economic: income/wealth/economic status,No -> Yes
9977,92436_200509-360693-2,"It is necessary to expand the lustration, to d...",secret collaborators,economic: occupation/profession,Yes -> No
10133,92712_199110-359840-1,A professional group that sees the enormous in...,A professional group that sees the enormous in...,non-economic: shared values/mentalities,Yes -> No
10350,93712_199611-377435-1,Within these forms of agricultural exploitatio...,the peasants,economic: class membership,No -> Yes


In [70]:
out.value_counts(['label', 'corrected_label']).unstack(fill_value=0)

corrected_label,No,Yes
label,,
No,2693,49
Yes,14,232


In [71]:
del out["_merge"]
del out["label_change"]

### output corrected annotations

In [72]:
out['label'] = out.apply(lambda row: row['corrected_label'] if not pd.isna(row['corrected_label']) else row['label'], axis=1)
del out['corrected_label']

In [73]:
# verify
out.label.value_counts(dropna=False)

label
No     10064
Yes      736
Name: count, dtype: int64

In [74]:
out.mention_id.nunique()

600

In [75]:
fp = data_path / 'final_corrected_annotations.tsv'
out.to_csv(fp, sep='\t', index=False)